In [ ]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Fri Oct 28 21:46:23 2016

@author: stephen
"""

from __future__ import print_function
import sys

sys.path.append("../")
import utils
import time
from tensorflow import keras
from keras.callbacks import EarlyStopping

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(813306)

start = time.time()


def readucr(filename):
    data = np.loadtxt(filename)
    Y = data[:, 0]
    X = data[:, 1:]
    return X, Y


epochs = 400

flist = ["../../datasets/GunPoint/GunPoint"]

for each in flist:
    fname = each
    x_train, y_train = readucr(fname + "_TRAIN.txt")
    x_test, y_test = readucr(fname + "_TEST.txt")
    nb_classes = len(np.unique(y_test))
    y_train = (
        (y_train - y_train.min()) / (y_train.max() - y_train.min()) * (nb_classes - 1)
    )
    y_test = (y_test - y_test.min()) / (y_test.max() - y_test.min()) * (nb_classes - 1)
    # batch_size = int(min(x_train.shape[0] / 10, 16))
    batch_size = int(x_train.shape[0])

    Y_train = keras.utils.to_categorical(y_train, nb_classes)
    Y_test = keras.utils.to_categorical(y_test, nb_classes)

    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean) / (x_train_std)

    # x_test_min = np.min(x_test, axis = 1, keepdims=1)
    # x_test_max = np.max(x_test, axis = 1, keepdims=1)
    x_test = (x_test - x_train_mean) / (x_train_std)

    # x_train = x_train.reshape(x_train.shape + (1,))
    # x_test = x_test.reshape(x_test.shape + (1,))

    x = keras.layers.Input(x_train.shape[1:])
    y = keras.layers.Dropout(0.1)(x)
    y = keras.layers.Dense(500, activation="relu")(y)
    y = keras.layers.Dropout(0.2)(y)
    y = keras.layers.Dense(500, activation="relu")(y)
    y = keras.layers.Dropout(0.2)(y)
    y = keras.layers.Dense(500, activation="relu")(y)
    y = keras.layers.Dropout(0.3)(y)
    out = keras.layers.Dense(nb_classes, activation="softmax")(y)

    model = keras.models.Model(inputs=x, outputs=out)

    optimizer = keras.optimizers.Adadelta()
    model.compile(
        loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
    )

    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor="loss", factor=0.5, patience=200, min_lr=0.1
    )

    es = EarlyStopping(monitor="val_loss", verbose=1, min_delta=0.001, patience=200)

    hist = model.fit(
        x_train,
        Y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        validation_data=(x_test, Y_test),
        callbacks=[reduce_lr, es],
    )
elapsed_time = time.time() - start
model.save("../data/MLP.h5")

In [ ]:
# evaluate the model
_, train_acc = model.evaluate(x_train, Y_train)
_, test_acc = model.evaluate(x_test, Y_test)

In [ ]:
utils.save_model_stats(
    "MLP",
    hist.history["loss"],
    hist.history["val_loss"],
    hist.history["accuracy"],
    hist.history["val_accuracy"],
    elapsed_time,
)